In [23]:
#前置作業準備，只要一次!!
df5k = pd.read_json("paras/mapping5K.json")

dfv = pd.read_csv("volumeData/20230105.csv")
dfv["id"] = dfv["id"].astype("string")
dfv = dfv.drop(columns=['close'])

import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

url4 = "https://www.wantgoo.com/investrue/all-alive"
r3 = requests.get(url4, headers = headers).content
soup = BeautifulSoup(r3, "html.parser")
rr3 = soup.prettify()
dfn = pd.read_json(rr3)
# dfn = pd.json_normalize(rr3, "industries" ,['id','name','shortName'])
dfn = dfn[(dfn["id"]>="1101") & (dfn["id"]<="9999") & (dfn["id"].str.len() == 4)]
dfn = dfn.drop(columns=['type','country','url', 'industries'])
dfn["id"] = dfn["id"].astype("string")
# dfn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2080 entries, 315 to 2430
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2080 non-null   string
 1   name    2080 non-null   object
 2   market  2074 non-null   object
dtypes: object(2), string(1)
memory usage: 65.0+ KB


In [25]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
pd.set_option('display.expand_frame_repr', False)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}
url2 = "https://www.wantgoo.com/investrue/all-quote-info"

r = requests.get(url2, headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
df = pd.read_json(rr)
df = df.drop(columns=['tradeDate', 'time','millionAmount'])
df["id"] = df["id"].astype("string")
df = pd.merge(df, dfn, left_on="id", right_on="id")  ## 結合股票名稱
df["float"] = df["close"].astype("float")
df["amp"] = (df["close"] - df["open"])/df["open"] * 100
df["jump"] = df["open"] - df["previousClose"]
df["jumpRate"] = df["jump"] / df["open"] * 100
df = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["id"].str.len() == 4) & (df["close"] > df["previousClose"])].sort_values("jumpRate", ascending=False)
# df.info()
# print(df.head(50))

url3 = "https://www.wantgoo.com/stock/all-turnover-rates"
r2 = requests.get(url3, headers = headers).content
soup = BeautifulSoup(r2, "html.parser")
rr2 = soup.prettify()
dfa = pd.read_json(rr2)
# print(dfa)

dfa["investrueId"] = dfa["investrueId"].astype("string")
dfb = pd.merge(df, dfa, left_on="id", right_on="investrueId")
dfb = dfb.drop(columns=['investrueId'])
dfb = dfb[(dfb["value"] > 1) ].sort_values("value", ascending=False)

### 計算量比參數 ###
import time
localtime = time.localtime()
nowTime = time.strftime("%I:%M", localtime)
# print(nowTime)
# nowTime = "09:13"
Rate5k = df5k[(df5k["e"] >= nowTime) & (df5k["b"] <= nowTime)]["w"].values[0]  #五分K量比預估放大參數
print(f"目前時間：{nowTime},預估量放大係數:{Rate5k}")

dfc = pd.merge(dfb, dfv, left_on="id", right_on="id")
dfc["volumeEst"] = dfc["volume"] * Rate5k                               ##預估量
dfc["volumeDiff"] = (dfc["volume"] - dfc["yVolume"]) / dfc["volume"] * 100
print(dfc.sort_values("value", ascending=False).head(50))


######################################################################################################
# import _beowFmt as fm 
# dd = dfb[["id","close","volume"]].sort_values("id")
# ss = 'id,close,yVolume\n'
# for d in dd.values.tolist():
#   ss += f"{d[0]},{d[1]},{d[2]}\n"
# fm.write_LogFile(f"volumeData/20230105.csv", ss)
## todo ##
# https://www.wantgoo.com/investrue/all-alive       #股票名稱
# https://www.wantgoo.com/investrue/all-previous-weekly-candlestick?v=1672966860000  上一周收盤價


目前時間：03:42,預估量放大係數:1.0
      id    flat   floor    ceil    open    high     low   close  previousClose  volume   float       amp  jump  jumpRate     value   name  market  yVolume  volumeEst  volumeDiff
0   8171   51.80   46.65   56.90   53.60   56.50   52.80   55.10          51.80   21551   55.10  2.798507  1.80  3.358209  32.29910     天宇     OTC    21551    21551.0         0.0
1   3479   83.00   74.70   91.30   84.10   90.00   83.70   88.20          83.00   17233   88.20  4.875149  1.10  1.307967  24.21652     安勤     OTC    17233    17233.0         0.0
2   4968  119.50  108.00  131.00  120.50  130.50  120.50  126.00         119.50   16412  126.00  4.564315  1.00  0.829876  18.20140     立積  Listed    16412    16412.0         0.0
3   3272   50.30   45.30   55.30   50.50   54.30   50.50   51.00          50.30    9797   51.00  0.990099  0.20  0.396040  17.93852     東碩     OTC     9797     9797.0         0.0
4   8044   59.40   53.50   65.30   61.10   65.00   60.60   64.00          59.40   

In [20]:


# multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')

,id,name,market
315,1101,台泥,Listed
317,1102,亞泥,Listed
318,1103,嘉泥,Listed
319,1104,環泥,Listed
320,1108,幸福,Listed
...,...,...,...
2426,9955,佳龍,Listed
2427,9957,燁聯,Emerging
2428,9958,世紀鋼,Listed
2429,9960,邁達康,OTC


In [ ]:
import _beowFmt as fm 

df2 = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["volume"] >= 1000)].sort_values(by=['id'], ascending=True)
dd = df2[{"id","volume"}]
ss = ''
for d in dd.values.tolist():
  ss += f"{d[1]}, {d[0]}\n"

fm.write_LogFile(f"keepData/stockVolume.csv", ss)

In [ ]:
while True:
    localtime = time.localtime()
    result = time.strftime("%Y-%m-%d %I:%M:%S %p", localtime)
    print(result)
    time.sleep(1)

In [ ]:
from pathlib import Path  

filepath = Path('https://www.wantgoo.com/investrue/all-quote-info')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

In [ ]:
# importing the requests library
import requests

# api-endpoint
URL = "http://maps.googleapis.com/maps/api/geocode/json"

# location given here
location = "delhi technological university"

# defining a params dict for the parameters to be sent to the API
PARAMS = {'address':location}

# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS)

# extracting data in json format
data = r.json()


# extracting latitude, longitude and formatted address
# of the first matching location
latitude = data['results'][0]['geometry']['location']['lat']
longitude = data['results'][0]['geometry']['location']['lng']
formatted_address = data['results'][0]['formatted_address']

# printing the output
print("Latitude:%s\nLongitude:%s\nFormatted Address:%s"
	%(latitude, longitude,formatted_address))
